# **Segunda entrega monografía**

**Integrantes:**
- Alan Stiven Camacho Restrepo
- Isabel Cristina Suárez Roldán.

In [2]:
# Importacion de librerias necesarias para el proyecto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Lista vacía para guardar los DataFrames
dfs = []

# Leer archivos del 0 al 10 y agregarlos a la lista
for i in range(11):
    file_name = f"base_variables_totales_parte{i}.parquet"
    df = pd.read_parquet(file_name)
    dfs.append(df)

In [4]:
# Tabla final: Unir todos los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)

In [5]:
# Tamaño de la tabla
df_final.shape

(1598523, 92)

In [6]:
df_final.head()

,fecha_orden,year,month,tipo_doc,cod_estado_cli,num_doc,cod_segm,dias_mora,desempleado,ingreso_final_smmlv,...,flag_estabilidad_6m_asto_total,flag_retiro_efectivo_bajo_ingreso_final,flag_retiro_efectivo_bajo_estimadores,cupo_smmlv,val_trx_cop_compra_smmlv,val_trx_cop_avance_smmlv,flag_avances_cupo_uso_tdc,razon_trx_max_una_cuota_ingreso_estimadores,flag_mal_uso_tdc_trx_max_ingreso_estimadores,valor_g
0,202104,2021,4,1,1,045b809be3,4,NaN,1.0,0.039338,...,1,0,0,None,None,None,0,None,0,G7
1,202402,2024,2,1,1,f25a890f5a,4,NaN,0.0,0.000000,...,1,0,0,None,None,None,0,None,0,G1
2,202408,2024,8,1,1,d58acc33de,M,178.0,NaN,0.000000,...,1,0,0,0.25384615384615385,None,None,0,None,0,G8
3,202210,2022,10,1,1,c1325d3a5b,4,0.0,NaN,0.480417,...,1,1,1,None,None,None,0,None,0,G5
4,202207,2022,7,1,1,1980a06c65,4,NaN,0.0,3.585892,...,1,0,0,None,None,None,0,None,0,G4


# Descripción de la tabla final

<h4> La tabla final es una base consolidada y anonimizada que resume el comportamiento financiero mensual de clientes del banco entre agosto de 2021 y marzo de 2025. Integra información de ingresos, gastos, productos financieros, uso de canales, estabilidad económica y riesgo crediticio. Contiene variables originales y derivadas, construidas a partir de 12 tablas cruzadas por cliente y fecha, con múltiples transformaciones, agregaciones y banderas (flags) para facilitar el análisis prescriptivo y la personalización del score crediticio.


In [7]:
def boxplots_univariados_multiples(df: pd.DataFrame, columnas: list, n_cols: int = 3) -> None:
    """
    Grafica múltiples boxplots para variables numéricas.

    Args:
        df (pd.DataFrame): DataFrame con los datos.
        columnas (list): Lista de nombres de columnas numéricas a graficar.
        n_cols (int): Número de columnas por fila en la visualización.
    """
    n = len(columnas)
    n_rows = -(-n // n_cols)  # Redondeo hacia arriba

    plt.figure(figsize=(5 * n_cols, 4 * n_rows))

    for i, col in enumerate(columnas, 1):
        plt.subplot(n_rows, n_cols, i)
        if pd.api.types.is_numeric_dtype(df[col]):
            sns.boxplot(y=df[col])
            plt.title(f"Boxplot: {col}")
            plt.tight_layout()
        else:
            plt.text(0.5, 0.5, f"{col}\n(no numérica)", ha='center', va='center')
            plt.axis('off')

    plt.suptitle("Boxplots univariados", fontsize=16, y=1.02)
    plt.show()


In [8]:
def graficas_univariadas_multiples(df: pd.DataFrame, columnas: list, n_cols: int = 3) -> None:
    """
    Grafica múltiples columnas univariadas (histogramas para numéricas, countplot para categóricas).

    Args:
        df (pd.DataFrame): DataFrame con los datos.
        columnas (list): Lista de nombres de columnas a graficar.
        n_cols (int): Número de columnas por fila en la visualización.
    """
    n = len(columnas)
    n_rows = -(-n // n_cols)  # Redondeo hacia arriba

    plt.figure(figsize=(5 * n_cols, 4 * n_rows))

    for i, col in enumerate(columnas, 1):
        plt.subplot(n_rows, n_cols, i)
        if df[col].dtype == "object" or df[col].nunique() < 20:
            sns.countplot(data=df, x=col, order=df[col].value_counts().index)
            plt.xticks(rotation=45)
            plt.title(f"Frecuencia: {col}")
        else:
            sns.histplot(df[col], bins=60, kde=False)
            plt.title(f"Distribución: {col}")
        plt.tight_layout()

    plt.suptitle("Gráficas univariadas", fontsize=16, y=1.1, x=0.3)
    plt.show()


In [9]:
columnas_no_importantes = ['fecha_orden', 'year', 'month', 'tipo_doc', 'cod_estado_cli', 'num_doc',
       'cod_segm']

In [10]:
columnas_no_utilizadas =[
    'fuente_ingreso_final_smmlv', 'supervivencia', 'probabilidad_contagio', 'potencial_contagio_mes',
    'susceptibilidad_mes', 'flag_regla_sobreendeudamiento', 'flag_regla_sobreendeudamiento_ce_ingreso_estimadores',
    'flag_regla_sobreendeudamiento_ci_ingreso_estimadores',
    'flag_regla_sobreendeudamiento_total_ingreso_estimadores', 'flag_regla_capacidad_pago', 'regla_razon_sobreendeudamiento',
    'regla_capacidad_pago'

]

In [11]:
columnas_dicotomicas = [
    'desempleado', 'flag_compras_basicas',
    'flag_compras_no_basicas', 'flag_inversiones',
    'flag_sobreendeudamiento_ce', 'flag_sobreendeudamiento_ci',
    'flag_sobreendeudamiento_total',
    'flag_capacidad_pago_ce', 'flag_capacidad_pago_ci',
    'flag_capacidad_pago_total',
    'flag_capacidad_pago_ce_estimadores',
    'flag_capacidad_pago_ci_estimadores',
    'flag_capacidad_pago_total_estimadores', 'estabilidad_mensual',
    'estabilidad_mensual_estimadores', 'flag_estabilidad_ingreso',
    'flag_estabilidad_ingreso_estimadores', 'flag_cuenta_cero',
    'flag_cliente_ahorra', 'flag_cliente_ahorra_estimadores',
    'flag_estabilidad_mensual_compras_basicas',
    'flag_estabilidad_mensual_compras_no_basicas',
    'flag_estabilidad_mensual_suma_basicas_no_basicas',
    'flag_estabilidad_mensual_gasto_total',
    'flag_estabilidad_6m_compras_basicas',
    'flag_estabilidad_6m_compras_no_basicas',
    'flag_estabilidad_6m_suma_basicas_no_basicas',
    'flag_estabilidad_6m_asto_total',
    'flag_retiro_efectivo_bajo_ingreso_final',
    'flag_retiro_efectivo_bajo_estimadores',
    'flag_avances_cupo_uso_tdc',
    'flag_mal_uso_tdc_trx_max_ingreso_estimadores'
]

In [12]:
columna_categorica = ["valor_g"]

In [13]:
columnas_numericas = list(set(df_final.columns) - (set(columnas_no_importantes) | set(columnas_no_utilizadas) | set(columnas_dicotomicas) | set(columna_categorica)))

In [ ]:
graficas_univariadas_multiples(
    df_final,
    columnas_numericas[:10],
    n_cols=5
    )

KeyboardInterrupt: 